In [1]:
import pandas as pd 
import requests 
import numpy as np 
import json, os, time, pdb 

In [2]:
file_dir = os.listdir('test_data')
urc_files = ['test_data/'+i for i in file_dir if 'urc_game' in i]
prem_files = [i for i in file_dir if 'prem_game' in i]
sr_files = [i for i in file_dir if 'sr_game' in i]

file_dir

['urc_game_stats_2023.csv',
 'urc_game_stats_2024.csv',
 'prem_game_stats_2021.csv',
 'sr_game_stats_2021.csv',
 'prem_game_stats_2024.csv',
 'sr_game_stats_2022.csv',
 'prem_only_2022_teams.csv',
 '.ipynb_checkpoints',
 'urc_game_stats_2022.csv',
 'prem_game_stats_2023.csv',
 'sr_game_stats_2024.csv',
 'prem_game_stats_2022.csv',
 'sr_only_2024_teams.csv',
 'urc_game_stats_2021.csv',
 'urc_only_2021_teams.csv',
 'sr_game_stats_2023.csv',
 'urc_only_2022_teams.csv']

In [3]:
df = pd.concat(list(map(pd.read_csv, urc_files)))

In [4]:
df.columns

Index(['game_id', 'league_id', 'home_team', 'home_team_score', 'away_team',
       'away_team_score', 'home_tries', 'away_tries', 'home_conversions',
       'away_conversions', 'home_penalty_goals', 'away_penalty_goals',
       'home_kick_percent', 'away_kick_percent', 'home_total_meters',
       'away_total_meters', 'home_kfh', 'away_kfh', 'home_pass_meters',
       'away_pass_meters', 'home_runs', 'away_runs', 'home_possession_1h_2h',
       'home_territory_1h_2h', 'home_clean_breaks', 'home_defenders_beaten',
       'home_offloads', 'home_rucks_won', 'home_mauls_won',
       'home_turnovers_conceeded', 'away_possession_1h_2h',
       'away_territory_1h_2h', 'away_clean_breaks', 'away_defenders_beaten',
       'away_offloads', 'away_rucks_won', 'away_mauls_won',
       'away_turnovers_conceeded', 'home_total_possession',
       'home_total_territory', 'away_total_possesion', 'away_total_territory',
       'home_scrum', 'home_lineout', 'away_scrum', 'away_lineout',
       'home_tackle

In [5]:
df['home_rucks_won']

0       56 / 61 (91%)
1      73 / 73 (100%)
2       91 / 94 (96%)
3      95 / 102 (93%)
4       74 / 77 (96%)
           ...       
93      86 / 92 (93%)
94    121 / 124 (97%)
95      87 / 90 (96%)
96         0 / 0 (0%)
97    111 / 117 (94%)
Name: home_rucks_won, Length: 438, dtype: object

In [6]:
poss_str = df['home_possession_1h_2h'].iloc[0]

In [7]:
test_str = df['home_rucks_won'].iloc[0]

In [8]:
poss_str[:poss_str.find('/')]

'59% '

In [11]:
test_str[:test_str.find('/')-1]

'56'

In [12]:
def slice_prop_func(x, char, break_space=True):
    
    if break_space == True:
        space = 1
    else:
        space = 0
    
    if char == '/':
        return int(x[:x.find(char)-space])
    elif char == '(':
         return int(x[x.find('/')+space:x.find('(')-space])  
    elif char == ')':
        return int(x[x.find('(')+space:x.find(')')-space]) * 0.01


def slice_poss_terr_func(x):
    two_h_val = int(x[x.find('/')+1:].replace('%', '')) * 0.01
    one_h_val = int(x[:x.find('/')].replace('%', '')) * 0.01
    return one_h_val, two_h_val

    

In [13]:
slice_variable_names = ['1h_poss', '2h_poss', '1h_terr', '2h_terr']

In [14]:
df['home_1h_poss'], df['home_2h_poss'] = df['home_possession_1h_2h'].apply(lambda x: slice_poss_terr_func(x)[0]), \
                                                df['home_possession_1h_2h'].apply(lambda x: slice_poss_terr_func(x)[1])

df['home_1h_terr'], df['home_2h_terr'] = df['home_territory_1h_2h'].apply(lambda x: slice_poss_terr_func(x)[0]), \
                                                df['home_territory_1h_2h'].apply(lambda x: slice_poss_terr_func(x)[1])

df['away_1h_poss'], df['away_2h_poss'] = df['away_possession_1h_2h'].apply(lambda x: slice_poss_terr_func(x)[0]), \
                                                df['away_possession_1h_2h'].apply(lambda x: slice_poss_terr_func(x)[1])

df['away_1h_terr'], df['away_2h_terr'] = df['away_territory_1h_2h'].apply(lambda x: slice_poss_terr_func(x)[0]), \
                                                df['away_territory_1h_2h'].apply(lambda x: slice_poss_terr_func(x)[1])

In [44]:
df[['home_1h_poss', 'home_2h_poss']]

,home_1h_poss,home_2h_poss
0,0.59,0.37
1,0.40,0.48
2,0.54,0.40
3,0.42,0.59
4,0.53,0.57
...,...,...
93,0.56,0.51
94,0.63,0.67
95,0.47,0.64
96,0.00,0.00


In [ ]:
prop_variable_names = {
    'rucks_won': 'rucks_won_num', 'rucks_total_num', 'rucks_won_percent',
    'mauls_won': 'mauls_won_num', 'mauls_total_num', 'mauls_won_percent', 
    'scrums_won': 'scrum_won_num', 'scrum_total_num', 'scrum_won_percent', 
    'lineouts_won': 'lineouts_won_num', 'lineouts_total_num', 'lineouts_won_percent'
}



In [45]:
df[['home_scrum']]

,home_scrum
0,9/15 (60%)
1,10/13 (76%)
2,6/6 (100%)
3,5/6 (83%)
4,6/10 (60%)
...,...
93,6/9 (66%)
94,7/8 (87%)
95,5/7 (71%)
96,0/0 (NaN%)


In [40]:
df['home_rucks_won_num'] = df['home_rucks_won'].apply(slice_prop_func, args=('/', ))
df['home_rucks_total_num'] = df['home_rucks_won'].apply(slice_prop_func, args=('(',))
df['home_rucks_won_percent'] = df['home_rucks_won'].apply(slice_prop_func, args=(')',))

In [41]:
df[['home_rucks_won', 'home_rucks_won_num', 'home_rucks_total_num', 'home_rucks_won_percent']]

,home_rucks_won,home_rucks_won_num,home_rucks_total_num,home_rucks_won_percent
0,56 / 61 (91%),56,61,0.91
1,73 / 73 (100%),73,73,1.00
2,91 / 94 (96%),91,94,0.96
3,95 / 102 (93%),95,102,0.93
4,74 / 77 (96%),74,77,0.96
...,...,...,...,...
93,86 / 92 (93%),86,92,0.93
94,121 / 124 (97%),121,124,0.97
95,87 / 90 (96%),87,90,0.96
96,0 / 0 (0%),0,0,0.00


In [13]:
df[['home_tackles']]

,home_tackles
0,139/156
1,210/229
2,186/205
3,214/228
4,105/126
...,...
93,138/158
94,75/88
95,95/107
96,0/0


In [15]:
df['home_tackles'].iloc[0]

'139/156'

In [14]:
df.columns

Index(['game_id', 'league_id', 'home_team', 'home_team_score', 'away_team',
       'away_team_score', 'home_tries', 'away_tries', 'home_conversions',
       'away_conversions', 'home_penalty_goals', 'away_penalty_goals',
       'home_kick_percent', 'away_kick_percent', 'home_total_meters',
       'away_total_meters', 'home_kfh', 'away_kfh', 'home_pass_meters',
       'away_pass_meters', 'home_runs', 'away_runs', 'home_possession_1h_2h',
       'home_territory_1h_2h', 'home_clean_breaks', 'home_defenders_beaten',
       'home_offloads', 'home_rucks_won', 'home_mauls_won',
       'home_turnovers_conceeded', 'away_possession_1h_2h',
       'away_territory_1h_2h', 'away_clean_breaks', 'away_defenders_beaten',
       'away_offloads', 'away_rucks_won', 'away_mauls_won',
       'away_turnovers_conceeded', 'home_total_possession',
       'home_total_territory', 'away_total_possesion', 'away_total_territory',
       'home_scrum', 'home_lineout', 'away_scrum', 'away_lineout',
       'home_tackle

In [6]:
dfs['away_won_ruck_per'] = dfs['away_rucks_won']/(dfs['away_rucks_won'] + dfs['home_rucks_won'])

TypeError: unsupported operand type(s) for /: 'str' and 'str'